imports

In [1]:
from __future__ import print_function, division
from builtins import range

import os
import sys,re
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, GlobalMaxPool1D, SpatialDropout1D, Conv1D
from tensorflow.keras.layers import LSTM, MaxPooling1D, Embedding, Dropout, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
# import tensorflow.keras.backend as K
from sklearn.metrics import roc_auc_score

In [ ]:
# from google.colab import drive

# drive.mount("/content/gdrive")

some configuration

In [2]:
MAX_SEQUENCE_LENGTH = 222
MAX_VOCAB_SIZE = 30000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 64
EPOCHS = 3

load in pre-trained word vectors

In [3]:
word2vec = {}
with open(os.path.join('/home/hassan/Documents/GEC_/toxic/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec


In [ ]:
print('Found %s word vectors.' % len(word2vec))

In [ ]:
# !python -m spacy download en_core_web_sm
# import spacy

# nlp = spacy.load("en_core_web_sm")

prepare text samples and their labels <br/>


In [4]:
import time
start_time = time.time()

sentences = np.empty(0,dtype='object')
targets = np.zeros((0,222),dtype='int32')

f=open('/home/hassan/Documents/GEC_/toxic/conll14st-preprocessed.m2')
line = f.readline()
while line:
    line = line.strip()
    if line:
        if re.search("^S",line):
            line = line[2:]
            lenline = len(line.split())
            a=np.empty(1,dtype='object')
            a[0]=line
            sentences=np.concatenate([sentences,a])
            b=np.zeros((1,222),dtype='int32')
            
            targets=np.concatenate([targets,b])
#             i = i + 1
#         elif re.search("^A",line):
#             # found annotation
#             for key in labels:
#                 if re.search(key,line):
#                     if lenline > int(line[2:4]):
#                         targets[i-lenline+int(line[2:4])]=int(labels.get(key))
#                     else:
#                         targets[i-lenline+(int(line[2:4])-1)]=int(labels.get(key))
#                     #print(words)
#                     #print(targets)
#                     #print(pos)
#                     break
    line=f.readline()
print("--- %s seconds ---" % (time.time() - start_time))


--- 639.4278144836426 seconds ---


In [5]:
print(sentences[3])
print(sentences.shape)
# print(targets.shape)

This will , if not already , caused problems as there are very limited spaces for us .
(57151,)


convert the sentences (strings) into integers

In [6]:
tokenizer = Tokenizer( num_words=MAX_VOCAB_SIZE, filters='\t\n')
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

word2idx = tokenizer.word_index

In [21]:
print("sequence[3]: ", sequences[3])
print(type(sequences))

print("max sequence length:", max(len(s) for s in sequences))
print("min sequence length:", min(len(s) for s in sequences))
for s in sequences:
    if len(s)==222:
        print(s)
# s = sorted(len(s) for s in sequences)
# print("median sequence length:", s[len(s) // 2])
#
# print("max word index:", max(max(seq) for seq in sequences if len(seq) > 0))

print('Found %s unique tokens.' % len(word2idx))
# print(next(iter(word2idx)))

sequence[3]:  [24, 23, 2, 74, 20, 504, 2, 361, 96, 15, 48, 16, 127, 202, 3939, 12, 206, 3]
<class 'list'>
max sequence length: 222
min sequence length: 1
[67, 161, 456, 2, 37, 2, 21, 116, 32, 19, 11162, 130, 11163, 95, 8232, 85, 11164, 85, 7380, 85, 7380, 85, 1188, 117, 77, 113, 19, 32, 19, 11162, 130, 11163, 95, 8232, 85, 11164, 85, 7380, 85, 7380, 85, 1188, 78, 1372, 85, 15022, 85, 9422, 85, 25073, 85, 9423, 85, 15023, 85, 15024, 85, 15025, 85, 15026, 85, 15027, 85, 15028, 85, 9422, 85, 15029, 85, 9423, 85, 25074, 85, 15030, 85, 25075, 1521, 85, 4422, 85, 15031, 85, 4422, 85, 15032, 85, 4422, 85, 15033, 85, 15034, 85, 4422, 85, 25076, 25077, 85, 4422, 85, 10587, 703, 85, 4422, 85, 15035, 85, 4422, 85, 15036, 85, 25078, 95, 15037, 95, 15038, 95, 15039, 95, 15040, 95, 25079, 25080, 95, 15041, 95, 15042, 95, 25081, 95, 25082, 25083, 95, 15043, 95, 15044, 95, 8232, 85, 15045, 85, 8024, 85, 8024, 85, 1372, 85, 15046, 85, 6623, 85, 15047, 85, 9424, 85, 15048, 85, 15049, 384, 58, 384, 187, 

In [10]:
import time
start_time = time.time()

labels = { "ArtOrDet":2,"Nn":3,"Vt":4,"Prep":5,"Vform":6,"Wform":7,"SVA":8 }
i=-1
f=open('/home/hassan/Documents/GEC_/toxic/conll14st-preprocessed.m2')
line = f.readline()
while line:
    line = line.strip()
    if line:
        if re.search("^S",line):
            i = i + 1
            for j in range(len(sequences[i])):
                targets[i,222-len(sequences[i])+j] = 1
        elif re.search("^A",line):
            # found annotation
            for key in labels:
                if re.search(key,line):
                    if len(sequences[i]) > int(line[2:4]):
                        targets[i,222-len(sequences[i])+int(line[2:4])]=int(labels.get(key))
                    else:
                        targets[i,222-len(sequences[i])+(int(line[2:4])-1)]=int(labels.get(key))
                    break
    line=f.readline()
print("--- %s seconds ---" % (time.time() - start_time))


--- 1.2189347743988037 seconds ---


In [20]:
print(targets[3])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 6 1 1 1 8 1 1 3 1 1 1]


pad sequences so that we get a N x T matrix

In [15]:

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [16]:
print(data[3])
print(data.shape)
print(data[1][0].shape)
print(type(data[1,0]))

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
ybin = lb.fit(targets)
lb.classes_


In [ ]:
targ = lb.transform(targets)
print(targ[45])
print(targ.shape)

In [ ]:
# del words
# del sequences
# del targets

In [ ]:
print("data[0]", data[0])
print('Shape of data tensor:', data.shape)

In [ ]:
x_training, x_test = data[:929254], data[929254:]
y_training, y_test = targ[:929254], targ[929254:]

In [ ]:
print(type(y_training))
print(y_training[45])

In [26]:
[word for word, index in word2idx.items() if index == 2]

[',']

prepare embedding matrix <br/>
Filling pre-trained embeddings...

In [ ]:
print("len word2idx", len(word2idx))
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
  if i < MAX_VOCAB_SIZE:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      # words not found in embedding index will be all zeros.
      embedding_matrix[i] = embedding_vector


In [ ]:
print("embedding_matrix shape:", embedding_matrix.shape)
print(word2idx.get("creating"))
print("embedding_matrix[1113]", embedding_matrix[1113])

load pre-trained word embeddings into an Embedding layer <br/>
note that we set trainable = False so as to keep the embeddings fixed <br/>
Building Embedding layer ...

In [ ]:
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  input_length=MAX_SEQUENCE_LENGTH,
  trainable=False
)

Building model...

In [ ]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(input_)
x = Dense(200, activation='relu')(x)
x = LSTM(128, dropout=0.2,return_sequences=True)(x)
# x = MaxPooling1D(3)(x)
# x = Dropout(0.2)(x)
# x = Conv1D(128, 3, activation='relu')(x)
# x = MaxPooling1D(3)(x)
# x = Dropout(0.2)(x)
x = LSTM(128, dropout=0.2, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
# x = Dropout(0.2)(x)
# x = Dense(128, activation='relu')(x)
output = Dense(8, activation='softmax')(x)
#this takes waay too much time
model = Model(input_, output)

In [ ]:
model.compile(
   loss='categorical_crossentropy',
   optimizer='adam',
   metrics=['accuracy']
)
# Display the model's architecture
model.summary()

In [ ]:
print('Training model...')
r = model.fit(
  x_training,
  y_training,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)

In [ ]:
e = model.evaluate(
    x_test,
    y_test
)

plot some data

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()


plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()


In [ ]:
p = model.predict(x_training)
aucs = []
for j in range(6):
    auc = roc_auc_score(y_training[:,j], p[:,j])
    aucs.append(auc)
print(np.mean(aucs))

In [ ]:
# scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (e[1]*100))

In [ ]:
model.save('/home/hassan/Documents/GEC_/GEC_classifie7C-2-.h5')